Projeto de Eletromagnetismo e Ondulatoria:

Membros:

- [André Frugis](https://github.com/andrefrugis): andref9@al.insper.edu.br
- [Felipe Frug Mauro](https://github.com/FelipeFrug): felipefm5@al.insper.edu.br
- [Joao Otavio Palma](https://github.com/joaoopalma): joaoogp@al.insper.edu.br
- [Lucas Hix](https://github.com/Peng1104): lucash@al.insper.edu.br

In [ ]:
# bibliotecas para o Projeto

import numpy as np